# Exercice 2 

## Regression Longstaff-Schwartz

On commence par importer les librairies dont nous avons besoin

In [13]:
import numpy as np
import time 
from scipy.stats import norm

On réutilise la fonction de simulations Monte-Carlo de spots du sous-jacent 

In [15]:
def simulate_bs_process(S0=50., r=0.05, sigma=0.15, T=1., h=1./360):
    nb_sim = int(T / h) - 1
    
    if nb_sim <= 0:
        return [S0]
    
    gauss_mean = (r - 0.5 * sigma**2) * h
    gauss_stddev = sigma * np.sqrt(h)
    
    gauss_rvs = np.random.normal(
        loc=gauss_mean,
        scale=gauss_stddev,
        size=nb_sim)
    
    exp_gauss = np.exp(gauss_rvs)
    exp_gauss = np.insert(exp_gauss, 0, S0)

    return np.cumprod(exp_gauss)   

Nous posons une matrice contenant l'ensemble des simulations de spot

In [16]:
nb_paths = 10000

time_vec = np.linspace(0., 1., 360)
sim_paths = np.array([simulate_bs_process() for _ in range(nb_paths)])

## Algorithme de Longstaff-Schwartz


In [24]:
start_time = time.clock()
n = 360 #Nombre de découpes de mon intervalle
spot = sim_paths
price= np.zeros([sim_paths.shape[0],n])
maturity = 1 #Définition de la longueur de mon intervalle = durée de diffusion en année

dt = 1./360 #Taille d'un intervalle
nbSim = 10000 #Nombre de simulation Monte-Carlo

S0 = 50 #Niveau du spot en t=0
r = 0.05 #rendement annualisé
sigma = 0.15 #volatilité annualisée de mon sous-jacent
K = 49 #Strike de mon CALL


t = np.linspace(0,maturity,n) #Création d'un vecteur qui contient mes t avec un incrément de (maturity/n)
S = np.zeros([n]) #Création d'un vecteur de 0 qui contiendra mes S(t)
S[0] = S0 #J'initialise la première valeur de mon vecteur de sous-jacent

"""
for j in range(0,nb_paths): #Calcul de la valeur de mon CALL à maturité
    if (spot[j,n-1]-K)>0:
        price[j,n-1]=spot[j,n-1]-K
    else:
        price[j,n-1]=0
"""
price = np.where(spot>K, spot-K, 0)
for m in range(1,n): #m va me permettre de "redescendre" jusqu'au temps 0

        x_reg = spot[:,n-m-1] #Variables explicatives
        y_reg = np.exp(-r*dt)*price[:,n-m] #Variable expliquée
        coeff_reg = np.polyfit(x_reg, y_reg, 2)

        #Je calcule le prix de mon option juste un pas avant en actualisant sa valeur à l'aide du taux sans risque
        price[:,n-m-1]=(coeff_reg[2]+coeff_reg[1]*spot[:,n-m-1]+coeff_reg[0]*spot[:,n-m-1]**2)


        #On pourrait rendre nul les prix d'options négatifs obtenus 
        #mais cela a tendance à rendre un prix final supérieur de 5% à celui
        #obtenu avec les formules fermées
        #for j in range(0,nbSim):
         #   if (price[j,n-m-1])<0:
          #      price[j,n-m-1]=0

print(np.mean(price[:,0])) #Prix final de mon option (je prends la moyenne des valeurs de ma première colonne)
print (time.clock()-start_time, "secondes") #Affiche le temps nécessaire pour obtenir notre prix

4.846633889575721
0.7691917299989655 secondes


C:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:32: RankWarning: Polyfit may be poorly conditioned


On vérifie notre prix calculé grâce à Monte Carlo et Longstaff Schwartz par la simple formule de Black & Scholes.
On cherche à pricer un call européen basique

In [10]:
def bs_call_price(S=100., r=0.02, sigma=0.2, t=0., T=1., K=100.):
    ttm = T - t
    
    if ttm < 0:
        return 0.
    elif ttm == 0.:
        return np.maximum(S - K, 0.)

    vol = sigma * np.sqrt(ttm)

    d_minus = np.log(S / K) + (r - 0.5 * sigma**2) * ttm
    d_minus /= vol

    d_plus = d_minus + vol

    res = S * norm.cdf(d_plus)
    res -= K * np.exp(-r * ttm) * norm.cdf(d_minus)

    return res

In [14]:
bs_call_price(S0, r, sigma, 0, T, K)

4.892712596170217